In [66]:
# data manipulation and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for saving the pipeline
import joblib

# from Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Binarizer

# from feature-engine
from feature_engine.imputation import (
    MeanMedianImputer,
    CategoricalImputer,
    DropMissingData
)

from feature_engine.transformation import LogTransformer

from feature_engine.encoding import (
    OrdinalEncoder,
)

from  feature_engine.outliers import(
    OutlierTrimmer
)

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper


In [2]:
data = pd.read_csv('Bank_Campaign.csv', sep=';')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,subscribed
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [9]:
def replaceWNAN(df):
    for l in ['unknown','nonexistent']:
        df=df.replace(l, np.nan)
    return df
def binarize(df):
    df=df.replace('yes',1)
    df=df.replace('no',0)
    df=df.replace('cellular',1)
    df=df.replace('telephone',0)
    return df

In [11]:
data = replaceWNAN(data)
data = binarize(data)
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['poutcome', 'month','day_of_week','subscribed'], axis=1), # predictive variables
    data['subscribed'], # target
    test_size=0.2, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)

X_train.shape, X_test.shape

((32950, 17), (8238, 17))

In [31]:
def knn_imput(data,k_neighbors):
    imputer = KNNImputer(n_neighbors=k_neighbors)
    data_imputer = pd.DataFrame(imputer.fit_transform(data),columns=data.columns)
    return data_imputer

from sklearn.base import BaseEstimator, TransformerMixin
class KNN_Imputer(BaseEstimator, TransformerMixin):
    def __init__(self,k_neighbors=3):
        self.k_neighbors = k_neighbors
    def fit(self,X,Y):
        from sklearn.impute import KNNImputer
        self.imputer = KNNImputer(n_neighbors=self.k_neighbors)
        return self
    def transform(self,X):
        from pandas import DataFrame
        return DataFrame(self.imputer.fit_transform(X),columns=X.columns)

In [64]:
class Mapper(BaseEstimator, TransformerMixin):

    def __init__(self, variables, mappings):

        if not isinstance(variables, list):
            raise ValueError('variables should be a list')

        self.variables = variables
        self.mappings = mappings

    def fit(self, X, y=None):
        # we need the fit statement to accomodate the sklearn pipeline
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.variables:
            X[feature] = X[feature].map(self.mappings)

        return X

In [81]:
# categorical variables with NA in train set
DROP_OBS_WITH_NA = ['job','marital','education','housing','loan']

# variables to map
EDU_VARS = ['education']

# categorical variables to encode
CATEGORICAL_VARS = ['job','marital']

# variables to binarize
BINARIZE_VARS = ['pdays']

# variable mappings
EDU_MAPPINGS = {'illiterate':0,'basic.4y':1, 'basic.6y':2,
                 'basic.9y':3, 'high.school':4,'professional.course':5, 'university.degree':6}

# outliers handling
OUTLIERS_VARS = ['age','duration','campaign']

# variables to log transform
NUMERICALS_LOG_VARS = ['campaign']

# the selected variables
FEATURES = [
    'age',
    'education',
    'duration',
    'campaign',
    'pdays',
    'cons.conf.idx',
    'euribor3m',
    'nr.employed']

In [82]:
X_train = X_train[FEATURES]
X_test = X_test[FEATURES]

X_train.shape, X_test.shape

((32950, 8), (8238, 8))

In [83]:
# classifier
from xgboost import XGBClassifier

In [91]:
# set up the pipeline
BSP_pipe = Pipeline([

    # ===== IMPUTATION =====
    # impute categorical variables with string missing
    #('drop_observations', DropMissingData(
    #   variables=DROP_OBS_WITH_NA)),
    
    # knn imputation 'default variable'
    #('knn_imputation', KNN_Imputer()),

    
   

    # ==== VARIABLE TRANSFORMATION =====
    ('log', LogTransformer(variables=NUMERICALS_LOG_VARS)),
    
#     ('yeojohnson', YeoJohnsonTransformer(variables=NUMERICALS_YEO_VARS)),
    
    ('binarizer', SklearnTransformerWrapper(
        transformer=Binarizer(threshold=500), variables=BINARIZE_VARS)),
    # === outliers ===
    ('outliers', OutlierTrimmer(
        capping_method='iqr', tail='right', fold=1.5, variables=OUTLIERS_VARS)),
    
    # === mappers ===
    ('mapper_qual', Mapper(
        variables=EDU_VARS, mappings=EDU_MAPPINGS)),


    # == CATEGORICAL ENCODING

    # encode categorical and discrete variables using the target mean
    #('categorical_encoder', OrdinalEncoder(
    #   encoding_method='ordered', variables=CATEGORICAL_VARS)),
    
    
    ('scaler', MinMaxScaler())])

In [92]:
# train the pipeline
BSP_pipe.fit(X_train, y_train)

Pipeline(steps=[('log', LogTransformer(variables=['campaign'])),
                ('binarizer',
                 SklearnTransformerWrapper(transformer=Binarizer(threshold=500),
                                           variables=['pdays'])),
                ('outliers',
                 OutlierTrimmer(capping_method='iqr', fold=1.5,
                                variables=['age', 'duration', 'campaign'])),
                ('mapper_qual',
                 Mapper(mappings={'basic.4y': 1, 'basic.6y': 2, 'basic.9y': 3,
                                  'high.school': 4, 'illiterate': 0,
                                  'professional.course': 5,
                                  'university.degree': 6},
                        variables=['education'])),
                ('scaler', MinMaxScaler())])

In [96]:

#     ('selector', SelectFromModel(Lasso(alpha=0.001, random_state=0))),
#    ('Xgbost', XGBClassifier(eval_metric='error', use_label_encoder=False)),
#])